In [20]:
import os

import pandas as pd
from langchain.output_parsers import PydanticOutputParser
from langchain_mistralai.chat_models import ChatMistralAI

from dotenv import load_dotenv
from baker.parser import RecipeParser, MESSAGE, CHAT_PROMPT_TEMPLATE
from baker.schemas import Recipe

In [21]:
MESSAGE

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instructions', 'ingredients', 'steps'], template='\nWhat are the ingredients and their associated quantities as well as the steps to make the recipe described by the following {ingredients} and {steps} provided as raw text ?\n\nIn particular, please provide the following information:\n- The name of the recipe\n- The serving size\n- The ingredients and their associated quantities\n- The steps to make the recipe\n- Any additional comments\nas described in {format_instructions}\n\n'))

In [26]:
CHAT_PROMPT_TEMPLATE.format_prompt(ingredients="carrot", steps="boil the carrot", format_instructions=Recipe)

ChatPromptValue(messages=[HumanMessage(content="\nWhat are the ingredients and their associated quantities as well as the steps to make the recipe described by the following carrot and boil the carrot provided as raw text ?\n\nIn particular, please provide the following information:\n- The name of the recipe\n- The serving size\n- The ingredients and their associated quantities\n- The steps to make the recipe\n- Any additional comments\nas described in <class 'baker.schemas.Recipe'>\n\n")])

In [2]:
df = pd.read_json("data/recipes_v1_with_token_counts.json")
df.head()

,title,date,tags,introduction,ingredients,direction,output,ingredients_count,direction_count
0,Creamy Mashed Potatoes,2021-03-12,"[potato, side, cheesefare]",![Creamy Mashed Potatoes](/pix/creamy-mashed-p...,The quantities here are for about four adult p...,1. Peel and cut the potatoes into medium sized...,"{'title': 'Creamy Mashed Potatoes', 'ingredien...",93,184
1,Red Sauce (Ragu all'Italiana),2021-03-16,"[italian, sauce]",My great-grandma's red sauce. All purpose: goo...,- 1/3 lb salt pork - 2 lb chuck roast - 3 Clov...,"1. Mince the garlic, peel the carrot, peel and...","{'title': 'Red Sauce (Ragu all'Italiana)', 'in...",61,117
2,Turmeric Flatbread,2022-08-09,"[bread, turkish]",A great companion to Turkish Red Lentil Soup. ...,- 1 cup Wheat Flour (white or whole) - 1/2 Tbs...,"1. Combine flour, turmeric, salt and baking po...","{'title': 'Turmeric Flatbread', 'ingredients':...",30,90
3,Zurich-Style Meat Saute,2023-01-06,"[beef, cream, swiss, quick]","Originally called ""Züri Gschnätzlets"" (Zurich ...","- 600g\tBeef, finely sliced - 250g\tMushrooms,...",1. Fry the meat in butter at high heat until i...,"{'title': 'Zurich-Style Meat Saute', 'ingredie...",37,126
4,Tuscan Style Pork Roast,2021-03-10,"[italian, pork, roast]",![tuscan-style-pork-roast](/pix/tuscan-style-p...,- 1 pork Roast - 2-3 Tbsp fresh rosemary - 8 c...,1.\tPreheat oven to 275°F (135°C) 2.\tButterfl...,"{'title': 'Tuscan Style Pork Roast', 'ingredie...",38,473


In [3]:
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")

In [17]:
llm = ChatMistralAI(api_key=MISTRAL_API_KEY, model_name="open-mixtral-8x7b")
parser = PydanticOutputParser(pydantic_object=Recipe)

In [18]:
recipe_parser = RecipeParser(llm=llm, parser=parser)

In [ ]:
df["ingredients"][0]

In [19]:
output = await recipe_parser.run(ingredients=df["ingredients"][0], steps=df["direction"][0])
output

Recipe(name='Bacon and Cheese Mashed Potatoes', serving_size=4, ingredients=[Ingredient(name='potatoes', quantity=1000.0, unit='g'), Ingredient(name='milk', quantity=200.0, unit='ml'), Ingredient(name='mayonnaise', quantity=200.0, unit='ml'), Ingredient(name='cheese', quantity=100.0, unit='g'), Ingredient(name='garlic powder', quantity=None, unit=None), Ingredient(name='bacon', quantity=12.0, unit='strips'), Ingredient(name='butter', quantity=None, unit=None), Ingredient(name='green onions', quantity=None, unit=None), Ingredient(name='black pepper', quantity=None, unit=None), Ingredient(name='salt', quantity=None, unit=None)], steps=['Peel and cut the potatoes into medium sized pieces.', 'Put the potatoes in a pot with some water so that it covers the potatoes and boil them for about 20-30 minutes, or until the potatoes are soft.', 'About ten minutes before removing the potatoes from the boiling water, cut the bacon into little pieces and fry it.', 'Warm up the milk and mayonnaise.', '

In [28]:
output.dict()

{'name': 'Bacon and Cheese Mashed Potatoes',
 'serving_size': 4,
 'ingredients': [{'name': 'potatoes', 'quantity': 1000.0, 'unit': 'g'},
  {'name': 'milk', 'quantity': 200.0, 'unit': 'ml'},
  {'name': 'mayonnaise', 'quantity': 200.0, 'unit': 'ml'},
  {'name': 'cheese', 'quantity': 100.0, 'unit': 'g'},
  {'name': 'garlic powder', 'quantity': None, 'unit': None},
  {'name': 'bacon', 'quantity': 12.0, 'unit': 'strips'},
  {'name': 'butter', 'quantity': None, 'unit': None},
  {'name': 'green onions', 'quantity': None, 'unit': None},
  {'name': 'black pepper', 'quantity': None, 'unit': None},
  {'name': 'salt', 'quantity': None, 'unit': None}],
 'steps': ['Peel and cut the potatoes into medium sized pieces.',
  'Put the potatoes in a pot with some water so that it covers the potatoes and boil them for about 20-30 minutes, or until the potatoes are soft.',
  'About ten minutes before removing the potatoes from the boiling water, cut the bacon into little pieces and fry it.',
  'Warm up the m

In [ ]:
df.describe()

In [ ]:
df["ingredients_count"].sum(), df["direction_count"].sum()

In [ ]:
df["ingredients_count"].sum() + df["direction_count"].sum()

In [ ]:
df.to_json("data/recipes_v1_with_token_counts.json")